In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

# 데이터프레임을 읽어옵니다.
excel_file = pd.read_excel("./data/A 음식점(15,726)_new.xlsx", engine="openpyxl")

# 데이터셋 클래스를 정의합니다.
class CustomDataset(Dataset):
    def __init__(self, data, transform=None, target_transform=None):
        self.df = data  # 데이터프레임을 직접 사용
        self.sentence, self.label = self.whatisthis(self.df) # 데이터프레임에서 문장(sentence)와 라벨(label)을 읽어옴

    def whatisthis(self, excel_df):
        sentence = excel_df["SENTENCE"] # 데이터프레임의 문장(SENTENCE)데이터를 읽어옴
        # 개체명의 첫번째 부분만을 읽어옴
        # excel_df['개체명'].tolist() -> 개체명 데이터를 리스트로 변환
        # row가 NA가 아니라면 ,(콤마)단위로 나눈 뒤 첫번째 요소만 앞뒤 공백을 제거해서 label 변수에 저장
        # 그렇지 않다면, ""(공백)을 저장
        label = [str(row).split(",")[0].strip() if not pd.isna(row) else "" for row in excel_df["개체명"].tolist()]

        return sentence, label # sentence, label 반환

    def __len__(self):
        return len(self.df) # 데이터셋의 길이 반환

    def __getitem__(self, idx):
        # 인덱스에 해당하는 sentence, label 반환
        sentence = self.sentence[idx]
        label = self.label[idx]

        return sentence, label

In [ ]:
# 데이터셋 인스턴스 생성
custom_dataset = CustomDataset(excel_file)

# 데이터 로더를 생성
batch_size = 64  # 원하는 배치 크기로 설정
# 데이터셋을 배치만큼 묶어서 전달하고 순회할 때마다 데이터를 섞음
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

sentence_datas = []
label_datas = []
# 데이터 로더를 사용하여 데이터를 반복(iterate)하면서 모델을 학습할 수 있습니다.
for batch in data_loader:
    sentences, labels = batch
    # 여기에서 모델 학습 루프를 구현하면 됩니다.
    for sentence, label in zip(sentences, labels): # zip() : 같은 인덱스의 항목끼리 묶음
        sentence_datas.append(sentence)
        label_datas.append(label)

        print("Sentence:", sentence)
        print("Label:", label)

In [ ]:
label_datas = set(label_datas)
label_datas.remove('')

In [ ]:
label2idx = {label: idx for idx, label in enumerate(label_datas)}